# GM 2021/22 
# Final Project - Shape deformation with cages
## Nicholas Gazzo, 4498892

The notebook takes in input two meshes: a model *M* and a coarse mesh *C* containing *M* in its interior, namely the cage, and support the following operations:
- Computation of *cage coordinates* for all vertices of *M* wrt *C* (see below); 
- Rendering of the two meshes superimposed in the same canvas, *M* in solid and *C* in wire frame;
- Selection of vertices of *C* and displacement of such vertices in 3D space. The position of vertices of *M* must be recomputed by means of cage coordinates after each displacement of the control structure and the two meshes must be rendered accordingly. 

The notebook implements the algorithm Mean Value Algorithm for the computation of the *cage coordinates*, which is described in the following paper [Ju et al. 2005](https://www.cse.wustl.edu/~taoju/research/meanvalue.pdf) (look at Fig.4). The algorithm is executed for each vertex *x* of *M* and provides its edge coordinates wrt all vertices of *C*.

In [1]:
import numpy as np
import igl
import meshplot as mp
import ipywidgets as iw
import timeit

from scipy.spatial.transform import Rotation
from functools import partial

# Input data

Firstly we need to read both the character model M and the related cage model C. 

In [8]:
# Read Character
v, f = igl.read_triangle_mesh('data/deer.obj')
# Read Cage
cage_v,cage_f = igl.read_triangle_mesh('data/deer_cage.obj')
cage_e = igl.edges(cage_f)
# Read Sphere
with np.load('data/octa_sphere_5.npz') as npl:
    sphere_v, sphere_f = npl['v'], npl['f']

# Min and Max on cage
min_v = cage_v.min(axis=0)
max_v = cage_v.max()
cage_v -= min_v
cage_v /= max_v
v -= min_v
v /= max_v
center_v = cage_v.max(axis=0)[[0,2]] / 2
cage_v[:,[0,2]] -= center_v
v[:,[0,2]] -= center_v

# Copy used later to reset the transformation
old_cage = cage_v.copy()

Here we have the rendering of the two meshes superimposed in the same canvas, M in solid and C in wire frame.

In [9]:
# Plot Character
plot = mp.plot(v,f)
# Plot cage
plot_cage_points = plot.add_points(cage_v,shading={"point_color": "green","point_size":.2})
plot_cage_edges = plot.add_edges(cage_v,cage_e,shading={"line_color": "gray"})

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0065747…

# Weights Evaluation

The *generateWeight*s function also print a recap of the dimensionality of both the character model M and the cage model C, while also providing the time elapsed during the execution of the function.

In [4]:
def generateWeights(v,f,cage_v,cage_f,eps = 1e-11):
    print("Character Vertices: ",v.shape[0])
    print("Cage Vertices: ",cage_v.shape[0])
    start = timeit.default_timer()
    # Weights (Matrix #V x #Cage_V)
    weights = np.zeros((v.shape[0],cage_v.shape[0]),dtype=np.float32)
    # For each vertex vi of the character model M
    for i,vi in enumerate(v):
        # Distances from each vertex of the cage
        di = np.linalg.norm(cage_v-vi,axis=1)
        ui = (cage_v-vi)/np.row_stack(di)
        # For each face of the cage model C
        for fj in cage_f:
            if np.any(di[fj]) < eps:
                break
            lj = np.array([np.linalg.norm(ui[fj[k-1]]-ui[fj[(k+1)%3]]) for k in range(3)])
            thetaj = 2*np.arcsin(lj/2)
            h = np.sum(thetaj)/2
            if(np.pi-h < eps):
                # vi lies on t, use 2D barycentric coordinates
                weights[i][fj] = np.array([np.sin(thetaj[k])*di[fj[k-1]]*di[fj[(k+1)%3]] for k in range(3)])
                break
            cj = np.array([(2*np.sin(h)*np.sin(h-thetaj[k]))/(np.sin(thetaj[(k+1)%3])*np.sin(thetaj[k-1]))-1 for k in range(3)])
            det = np.linalg.det(ui[fj])
            sj = np.sign(det)*np.sqrt(np.abs(1-(cj**2)))
            if np.any(np.abs(sj) < eps):
                # Vi lies outside t on the same plane, ignore t
                continue
            weights[i][fj] += np.array([(thetaj[k]-cj[(k+1)%3]*thetaj[k-1]-cj[k-1]*thetaj[(k+1)%3])/(di[fj[k]]*np.sin(thetaj[(k+1)%3])*sj[k-1]) for k in range(3)])
    # Divide by sum of weights for each row
    weights /= np.row_stack(np.sum(weights,axis=1))
    stop = timeit.default_timer()
    print('Execution Time: %.2f sec.'%(stop - start))
    return weights

def deform(v,f,cage_v,cage_f,weights):
    for i in range(v.shape[0]):
        if np.sum(weights[i,:]) > 0:
            v[i] = np.dot(weights[i,:],cage_v)
    return v

In [5]:
weights = generateWeights(v,f,cage_v,cage_f)

print("Weights Matrix shape: ",weights.shape)

Character Vertices:  1130
Cage Vertices:  48


D:\Programmi\Anaconda\lib\site-packages\ipykernel_launcher.py:23: RuntimeWarning: invalid value encountered in double_scalars


Execution Time: 16.21 sec.
Weights Matrix shape:  (1130, 48)


# Interactive Tool

It seems that the meshplot library is not able to update points and lines added with the functions *add_points* and *add_edges* (which I'm using in order to draw the cage). So in order to solve this I implemented some functions to update the attributes of objects of this type. These functions simply replace the attributes stored inside the **__objects** array, while also providing a notification to the renderer that these objects need to be updated visually.

To support interaction I adapted the jupyter notebook provided as resource in the description ([Selection.ipynb](https://github.com/danielepanozzo/gp/blob/master/Assignment_5/Selection.ipynb)).

In [6]:
def update_points(viewer,oid,vertices=None,colors=None,colormap='viridis'):
    if type(vertices) != type(None):
        viewer._Viewer__objects[oid]["geometry"].attributes["position"].array = vertices
        viewer._Viewer__objects[oid]["geometry"].attributes["position"].needsUpdate = True
    if type(colors) != type(None):
        c = mp.utils.get_colors(colors, colormap, normalize=False).astype("float32")
        viewer._Viewer__objects[oid]["geometry"].attributes["color"].array = c
        viewer._Viewer__objects[oid]["geometry"].attributes["color"].needsUpdate = True
        
def update_edges(viewer,oid,vertices=None):
    if type(vertices) != type(None):
        viewer._Viewer__objects[oid]["geometry"].attributes["position"].array = vertices
        viewer._Viewer__objects[oid]["geometry"].attributes["position"].needsUpdate = True

To select either a single vertex or a group of vertices of cage *C* the user firstly need to move the sphere mesh, using the three sliders controls, and then to press the *Select* button, which will select all the vertices inside the ball.
The selected vertices will be highlighted with a different color (green for unselected vertices and red for selected vertices). It also possible to deselect all the currently selected vertices by pressing the *Clear Selection* button.

After selecting the vertices it is possible interact with them by shifting their position (by a chosen amount) using the related *+* and *-* buttons. The notebook will automaticaly update both the cage position and the character (by calculating the deformation). Lastly to reset the current deformation press the *Reset* button, which will bring back the cage vertices to their original positions.

In [7]:
# Keep track of the selected vertices for the cage
selected = np.full(cage_v.shape[0],False)
sf = { "coord":[] }
# Meshplot
paint_ui = mp.Viewer({})
# Plot Character
paint_character = paint_ui.add_mesh(v,f)
# Plot cage
paint_cage_points = paint_ui.add_points(cage_v,c=selected,shading={"colormap":"RdYlGn_r","point_size":.2})
paint_cage_edges = paint_ui.add_edges(cage_v,cage_e,shading={"line_color": "gray"})
# Plot sphere
paint_sphere = paint_ui.add_mesh(sphere_v*0.05, sphere_f, shading={"flat" : False},c=np.array([1,0,0]))

# Build the UI
# Selection Button
select_button = iw.Button(description="Select",)
clear_button = iw.Button(description="Clear Selection")
reset_button = iw.Button(description="Reset")

def select_clicked(b):
    slicer = np.where(np.linalg.norm(cage_v - sf["coord"][1:],axis=1) < sf["coord"][0])[0]
    selected[slicer] = True
    update_points(paint_ui, paint_cage_points, colors=selected, colormap="RdYlGn_r")
    
def clear_clicked(b):
    selected[:] = False
    update_points(paint_ui, paint_cage_points, colors=selected, colormap="RdYlGn_r")
    
def reset_clicked(b):
    cage_v[:] = old_cage.copy()
    update_points(paint_ui, paint_cage_points, vertices=cage_v)
    update_edges(paint_ui, paint_cage_edges, vertices=cage_v[cage_e.ravel()])
    paint_ui.update_object(oid = paint_character, vertices=deform(v,f,cage_v,cage_f,weights))
    
select_button.on_click(select_clicked)
clear_button.on_click(clear_clicked)
reset_button.on_click(reset_clicked)

buttons = iw.HBox([select_button, clear_button,reset_button])

# Sphere Movement
min_v = np.floor(cage_v.min(axis=0))
max_v = np.ceil(cage_v.max(axis=0))
sphere_radius = iw.FloatSlider(min=0, max=1, value=0.05, step=0.05, description="Radius")
sphere_x = iw.FloatSlider(min=min_v[0], max=max_v[0], value=0, step=0.05, description="Sphere x")
sphere_y = iw.FloatSlider(min=min_v[1], max=max_v[1], value=0, step=0.05, description="Sphere y")
sphere_z = iw.FloatSlider(min=min_v[2], max=max_v[2], value=0, step=0.05, description="Sphere z")

# Accordion Build
accordion_1 = iw.Accordion(children=[iw.VBox([sphere_radius,sphere_x,sphere_y,sphere_z])])
accordion_1.set_title(0,"Cage Vertices Selection")

def handle_slider_change(value):
    sf["coord"] = [sphere_radius.value,sphere_x.value,sphere_y.value,sphere_z.value]
    paint_ui.update_object(oid = paint_sphere, vertices = sphere_v*sf["coord"][0] + np.array(sf["coord"][1:]))
    
sphere_radius.observe(handle_slider_change, names='value')
sphere_x.observe(handle_slider_change, names='value')
sphere_y.observe(handle_slider_change, names='value')
sphere_z.observe(handle_slider_change, names='value')

# Cage Shift UI
def plus_clicked(coord,text,b):
    cage_v[selected,coord] += text.value
    update_points(paint_ui, paint_cage_points, vertices=cage_v)
    update_edges(paint_ui, paint_cage_edges, vertices=cage_v[cage_e.ravel()])
    paint_ui.update_object(oid = paint_character, vertices=deform(v,f,cage_v,cage_f,weights))
    
def minus_clicked(coord,text,b):
    cage_v[selected,coord] -= text.value
    update_points(paint_ui, paint_cage_points, vertices=cage_v)
    update_edges(paint_ui, paint_cage_edges, vertices=cage_v[cage_e.ravel()])
    paint_ui.update_object(oid = paint_character, vertices=deform(v,f,cage_v,cage_f,weights))
    
shift_list = []
for i,coord in enumerate(["x","y","z"]):
    plus_button = iw.Button(icon="plus")
    coord_text = iw.FloatText(value=0.1, description="Shift %s"%coord)
    minus_button = iw.Button(icon="minus")
    plus_button.on_click(partial(plus_clicked,i,coord_text))
    minus_button.on_click(partial(minus_clicked,i,coord_text))
    shift_list.append(iw.HBox([coord_text, plus_button, minus_button]))
accordion_2 = iw.Accordion(children=[iw.VBox(shift_list)])
accordion_2.set_title(0,"Cage Vertices Shift")


controller_ui = iw.VBox([buttons,accordion_1,accordion_2])

display(iw.HBox([paint_ui._renderer,controller_ui]))  